# Работа со строковыми значениями

__Автор задач: Блохин Н.В. (NVBlokhin@fa.ru)__

Материалы:
* Макрушин С.В. Лекция "Работа со строковыми значениям"
* https://pyformat.info/
* https://docs.python.org/3/library/re.html
    * https://docs.python.org/3/library/re.html#flags
    * https://docs.python.org/3/library/re.html#functions
* https://pythonru.com/primery/primery-primeneniya-regulyarnyh-vyrazheniy-v-python
* https://kanoki.org/2019/11/12/how-to-use-regex-in-pandas/
* https://realpython.com/nltk-nlp-python/

## Задачи для совместного разбора

1. Вывести на экран данные из словаря `obj` построчно в виде `k = v`, задав формат таким образом, чтобы знак равенства оказался на одной и той же позиции во всех строках. Строковые литералы обернуть в кавычки.

In [1]:
obj = {
    "home_page": "https://github.com/pypa/sampleproject",
    "keywords": "sample setuptools development",
    "license": "MIT",
}

2. Написать регулярное выражение,которое позволит найти номера групп студентов.

In [4]:
obj = pd.Series(["Евгения гр.ПМ19-1", "Илья пм 20-4", "Анна 20-3"])
obj

0    Евгения гр.ПМ19-1
1         Илья пм 20-4
2            Анна 20-3
dtype: object

3. Разбейте текст формулировки задачи 2 на слова.

## Лабораторная работа 6

### Форматирование строк

1\. Загрузите данные из файла `recipes_sample.csv` (__ЛР2__) в виде `pd.DataFrame` `recipes` При помощи форматирования строк выведите информацию об id рецепта и времени выполнения 5 случайных рецептов в виде таблицы следующего вида:

    
    |      id      |  minutes  |
    |--------------------------|
    |    61178     |    65     |
    |    202352    |    80     |
    |    364322    |    150    |
    |    26177     |    20     |
    |    224785    |    35     |
    
Обратите внимание, что ширина столбцов заранее неизвестна и должна рассчитываться динамически, в зависимости от тех данных, которые были выбраны. 

In [1]:
import pandas as pd

# Загрузка данных из файла в DataFrame
recipes = pd.read_csv('recipes_sample.csv')

# Выбор случайных 5 рецептов
random_recipes = recipes.sample(n=5)

# Форматирование строк
formatted_recipes = random_recipes[['id', 'minutes']].astype(str)

# Вывод таблицы
table = formatted_recipes.to_string(index=False)
print(table)


    id minutes
229062     110
492741      60
113385      35
 96517      30
 33081     180


2\. Напишите функцию `show_info`, которая по данным о рецепте создает строку (в смысле объекта python) с описанием следующего вида:

```
"Название Из Нескольких Слов"

1. Шаг 1
2. Шаг 2
----------
Автор: contributor_id
Среднее время приготовления: minutes минут
```

    
Данные для создания строки получите из файлов `recipes_sample.csv` (__ЛР2__) и `steps_sample.xml` (__ЛР3__). 
Вызовите данную функцию для рецепта с id `170895` и выведите (через `print`) полученную строку на экран.

In [2]:
from bs4 import BeautifulSoup
with open('steps_sample.xml') as f:
    data = BeautifulSoup(f, 'xml')

In [3]:
def show_info(recipe_step):
    d = ''
    
    # Check if the recipe with the given id exists
    if len(recipes[recipes["id"] == recipe_step]) > 0:
        d += f'"{recipes[recipes["id"] == recipe_step].iloc[0]["name"].title()}"\n\n'
        
        for i in data.find_all('recipe'):
            if int(i.find('id').get_text()) == recipe_step:
                count = 1
                for j in i.find_all('step'):
                    d += f'{count}. {j.get_text().capitalize()}\n'
                    count += 1
                    
        d += f'{"."*10}\nАвтор: {recipes[recipes["id"] == recipe_step].iloc[0]["contributor_id"]}\n'
    else:
        d = f'Рецепт с id {recipe_step} не найден.'
    
    return d

print(show_info(1700895))


Рецепт с id 1700895 не найден.


## Работа с регулярными выражениями

3\. Напишите регулярное выражение, которое ищет следующий паттерн в строке: число (1 цифра или более), затем пробел, затем слова: hour или hours или minute или minutes. Произведите поиск по данному регулярному выражению в каждом шаге рецепта с id 25082. Выведите на экран все непустые результаты, найденные по данному шаблону.

In [7]:
from bs4 import BeautifulSoup

def find_time_patterns(recipe_id):
    # Загрузка данных о шагах приготовления из файла steps_sample.xml
    with open('steps_sample.xml', 'r') as file:
        xml_data = file.read()

    # Парсинг XML с помощью BeautifulSoup
    soup = BeautifulSoup(xml_data, 'xml')

    # Поиск шагов приготовления для указанного рецепта
    steps = soup.find_all('step', id=str(recipe_id))

    # Поиск и вывод информации о времени в каждом шаге
    for step in steps:
        time_tags = step.find_all(text=['hour', 'hours', 'minute', 'minutes'])
        for time_tag in time_tags:
            time_value = time_tag.strip()
            print(time_value)

# Вызов функции find_time_patterns для рецепта с id 25082
recipe_id = 25082
find_time_patterns(recipe_id)


4\. Напишите регулярное выражение, которое ищет шаблон вида "this..., but" _в начале строки_ . Между словом "this" и частью ", but" может находиться произвольное число букв, цифр, знаков подчеркивания и пробелов. Никаких других символов вместо многоточия быть не может. Пробел между запятой и словом "but" может присутствовать или отсутствовать.

Используя строковые методы `pd.Series`, выясните, для каких рецептов данный шаблон содержится в тексте описания. Выведите на экран количество таких рецептов и 3 примера подходящих описаний (текст описания должен быть виден на экране полностью).

In [11]:
import pandas as pd

# Загрузка данных из файла recipes_sample.csv в DataFrame
recipes = pd.read_csv('recipes_sample.csv')

# Заполнение пропущенных значений в столбце 'description' пустой строкой
recipes['description'] = recipes['description'].fillna('')

# Формирование маски для поиска шаблона в начале строки
mask = recipes['description'].str.contains(r'^this\.{3}\s?,\s?but', regex=True, na=False)

# Фильтрация рецептов, удовлетворяющих шаблону
matching_recipes = recipes[mask]

# Вывод количества и примеров рецептов, удовлетворяющих шаблону
print("Количество рецептов, удовлетворяющих шаблону:", len(matching_recipes))
print("Примеры описаний рецептов:")
for description in matching_recipes['description'].head(3):
    print(description)


Количество рецептов, удовлетворяющих шаблону: 0
Примеры описаний рецептов:


5\. В текстах шагов рецептов обыкновенные дроби имеют вид "a / b". Используя регулярные выражения, уберите в тексте шагов рецепта с id 72367 пробелы до и после символа дроби. Выведите на экран шаги этого рецепта после их изменения.

In [18]:
import csv
import re

# Открываем файл recipes_sample.csv
with open('recipes_sample.csv', 'r', encoding='utf-8') as file:
    reader = csv.DictReader(file)
    recipes = list(reader)

# Идентификатор рецепта, который мы хотим изменить
recipe_id = 72367

# Находим указанный рецепт по его идентификатору
target_recipe = next((recipe for recipe in recipes if int(recipe['id']) == recipe_id), None)

if target_recipe:
    # Получаем шаги рецепта
    recipe_steps = target_recipe['n_steps']

    # Убираем пробелы до и после символа дроби
    updated_recipe_steps = re.sub(r'\s*/\s*', '/', recipe_steps)

    # Выводим измененные шаги рецепта
    print(updated_recipe_steps)
else:
    print(f"Рецепт с идентификатором {recipe_id} не найден.")


12.0


### Сегментация текста

6\. Разбейте тексты шагов рецептов на слова при помощи пакета `nltk`. Посчитайте и выведите на экран кол-во уникальных слов среди всех рецептов. Словом называется любая последовательность алфавитных символов (для проверки можно воспользоваться `str.isalpha`). При подсчете количества уникальных слов не учитывайте регистр.

In [21]:
import nltk
nltk.download('punkt')

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\mehro\AppData\Roaming\nltk_data...
[nltk_data]   Unzipping tokenizers\punkt.zip.


True

In [22]:
import csv
import nltk
from nltk.tokenize import word_tokenize

# Открываем файл recipes_sample.csv
with open('recipes_sample.csv', 'r', encoding='utf-8') as file:
    reader = csv.DictReader(file)
    recipes = list(reader)

# Инициализируем множество для хранения уникальных слов
unique_words = set()

# Обходим все рецепты
for recipe in recipes:
    # Получаем текст шагов рецепта
    recipe_steps = recipe['n_steps']

    # Токенизируем текст на слова
    words = word_tokenize(recipe_steps)

    # Фильтруем слова, оставляя только алфавитные символы
    words = [word.lower() for word in words if word.isalpha()]

    # Добавляем уникальные слова во множество
    unique_words.update(words)

# Подсчитываем количество уникальных слов
num_unique_words = len(unique_words)

# Выводим количество уникальных слов на экран
print(f"Количество уникальных слов: {num_unique_words}")


Количество уникальных слов: 0


7\. Разбейте описания рецептов из `recipes` на предложения при помощи пакета `nltk`. Найдите 5 самых длинных описаний (по количеству _предложений_) рецептов в датасете и выведите строки фрейма, соответствующие этим рецептами, в порядке убывания длины.

In [23]:
import csv
import nltk
from nltk.tokenize import sent_tokenize

# Открываем файл recipes_sample.csv
with open('recipes_sample.csv', 'r', encoding='utf-8') as file:
    reader = csv.DictReader(file)
    recipes = list(reader)

# Добавляем новый столбец 'sentences' для хранения предложений рецептов
for recipe in recipes:
    recipe['sentences'] = sent_tokenize(recipe['description'])

# Сортируем рецепты по количеству предложений в описании в порядке убывания
sorted_recipes = sorted(recipes, key=lambda x: len(x['sentences']), reverse=True)

# Выводим пять самых длинных описаний рецептов
for recipe in sorted_recipes[:5]:
    print(recipe['id'], recipe['description'])
    print("Number of sentences:", len(recipe['sentences']))
    print("---")


334113 this wonderful icing is used for icing cakes and cookies as well as for borders and art work on cakes.  it makes a delicious filling also between the layers of cakes and under fondant icing.  you can make roses but it takes 3 or more days to dry them depending on the humidity. 

there are many versions of “buttercream” icing. some are made with eggs and all butter.  some varieties, you have to cook your sugar to a softball stage.  others are 100% shortening or a combination of shortening and butter.

each decorator has his or her favorite.  i personally think that the best taste and textured recipe is the one that has you cook your sugar, add to whipped eggs and use pounds of butter per batch. but…. i live in a state that can easily be a 100 degrees for days on end during the summer and you know what butter does on hot days.  it melts!  a greasy puddle of melted icing on a cake plate is not something i want to look at or eat.  

your top notch decorators have a few options we do

8\. Напишите функцию, которая для заданного предложения выводит информацию о частях речи слов, входящих в предложение, в следующем виде:
```
PRP   VBD   DT      NNS     CC   VBD      NNS        RB   
 I  omitted the raspberries and added strawberries instead
``` 
Для определения части речи слова можно воспользоваться `nltk.pos_tag`.

Проверьте работоспособность функции на названии рецепта с id 241106.

Обратите внимание, что часть речи должна находиться ровно посередине над соотвествующим словом, а между самими словами должен быть ровно один пробел.


In [24]:
import nltk
from nltk.tokenize import word_tokenize
from nltk import pos_tag

def display_pos_tags(sentence):
    # Токенизируем предложение на слова
    words = word_tokenize(sentence)

    # Определяем части речи для каждого слова
    pos_tags = pos_tag(words)

    # Форматируем результаты в требуемом виде
    formatted_tags = []
    for word, tag in pos_tags:
        formatted_tags.append(f"{tag:<7}")
        formatted_tags.append(f"{word}")
    formatted_result = ' '.join(formatted_tags)

    return formatted_result


# Заданное предложение для проверки
sentence = "I omitted the raspberries and added strawberries instead"

# Вызываем функцию для заданного предложения
result = display_pos_tags(sentence)

# Выводим результат на экран
print(result)

LookupError: 
**********************************************************************
  Resource [93maveraged_perceptron_tagger[0m not found.
  Please use the NLTK Downloader to obtain the resource:

  [31m>>> import nltk
  >>> nltk.download('averaged_perceptron_tagger')
  [0m
  For more information see: https://www.nltk.org/data.html

  Attempted to load [93mtaggers/averaged_perceptron_tagger/averaged_perceptron_tagger.pickle[0m

  Searched in:
    - 'C:\\Users\\mehro/nltk_data'
    - 'C:\\Users\\mehro\\anaconda3\\nltk_data'
    - 'C:\\Users\\mehro\\anaconda3\\share\\nltk_data'
    - 'C:\\Users\\mehro\\anaconda3\\lib\\nltk_data'
    - 'C:\\Users\\mehro\\AppData\\Roaming\\nltk_data'
    - 'C:\\nltk_data'
    - 'D:\\nltk_data'
    - 'E:\\nltk_data'
**********************************************************************
